In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

In [3]:
CONN_SCHEMA='sbx_team_digitcamp'

In [4]:
%%time
deals = \
hive.sql(''' select 
                    inn, 
                    create_dt,
                    complete_dt,
                    host_prod_id,
                    close_reason_type_name,
                    appl_stage_name

             from {}.{} where (inn = '2221191879') and
                              (creation_month in ('2021-02-01', 
                                                  '2021-03-01',
                                                  '2021-04-01',
                                                  '2021-05-01') )'''.format(CONN_SCHEMA, "MA_CMDM_MA_DEAL_NEW"))

CPU times: user 1.68 ms, sys: 1.56 ms, total: 3.24 ms
Wall time: 3.28 s


In [5]:
%%time
deals.count()

CPU times: user 301 ms, sys: 501 ms, total: 801 ms
Wall time: 39min 41s


3

In [7]:
load_table(SBX_TEAM_DIGITCAMP, 'tmp_visit_part', hive).show()

,visitNumber,cid,commonSegmentoUID,sbbolUserId,hitPagePath,hitPageTitle,sessionStartTime,sessionDate,hitType,hitNumber,hitTime,eventCategory,eventAction,eventLabel,hitPageHostName,ctl_loading,sessionId,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity
0,1248,1452727868.1587947876,None,None,/ic/dcb/,СберБизнес,1628897025,2021-08-14,EVENT,3,0,[std]: login,show,[news]: [show],localhost,11002249,1628897025,False,None,desktop,Khabarovsk Krai,Khabarovsk
1,2,2143600002.1628852233,None,None,/main,СберБанк Онлайн,1628945635,2021-08-14,PAGE,28,100685,None,None,None,web1.online.sberbank.ru,11002249,1628945635,False,None,desktop,Republic of Bashkortostan,Uchaly
2,1,1043955251.1628925029,pMchzHNegren,None,/help/business/sbbol/100296,Как создать платёжное поручение — СберБанк,1628925029,2021-08-14,EVENT,16,163562,SITE_Other_.help.business.sbbol.100296,informing_leads,scroll 50 percent,www.sberbank.ru,11002249,1628925029,False,None,desktop,Krasnodar Krai,Anapa
3,1581,556968546.1388994877,None,ec615264-6a3f-4fa6-8a87-f6e373fe4547,/main,СберБизнес,1628931602,2021-08-14,EVENT,20,26251,[operations]: offers,impression-story,[placement: main[main_feed]][container: a9ba93ee-d5c6-d10d-e054-90e2ba8333f9][bc: adv_non_bank][product: rabota_ru],sbi.sberbank.ru,11002249,1628931602,False,None,desktop,Vologda Oblast,Vologda
4,480,1598249319.1589364527,None,8a3f5f37-6361-4c1f-94fc-2b8f71d6cd03,/main,СберБизнес,1628925210,2021-08-14,EVENT,46,394568,[std]: layout,show,[layout.primarymenusbbol3]: [novelty_info],sbi.sberbank.ru,11002249,1628925210,False,None,desktop,Republic of Bashkortostan,Ufa
5,247,1333304315.1572956738,None,None,/payments/provider#/main,СберБанк Онлайн,1628940116,2021-08-14,PAGE,21,357979,None,None,None,web1.online.sberbank.ru,11002249,1628940116,True,SM-A305FN,mobile,Sakhalin Oblast,Yuzhno-Sakhalinsk
6,3,1061117300.1626119128,fbOtmfdaV4vn,None,/ru/legal/investments/globalmarkets/sbmarkets,Система электронной торговли SberCIB Terminal — СберБанк,1628971805,2021-08-14,PAGE,38,826865,None,None,None,www.sberbank.ru,11002249,1628971805,False,None,desktop,Saint Petersburg,Saint Petersburg
7,636,1107583888.1591630230,None,3c25f07c-09d4-fc84-e054-90e2ba85d0c8,/accounts-payments/account/:accountId#lightbox/payment/:type/:id,СберБизнес,1628945019,2021-08-14,EVENT,60,182900,[field]: payments,ruble_payment_edit__rub_comm,counterpartyaccountnumber,sbi.sberbank.ru,11002249,1628945019,False,None,desktop,Moscow Oblast,Dmitrov
8,22,2142242428.1628617076,None,None,/CSAFront/index.do,Вход - СберБанк Онлайн,1628950212,2021-08-14,PAGE,4,7806,None,None,None,online.sberbank.ru,11002249,1628950212,True,iPhone,mobile,Rostov Oblast,Rostov-on-Don
9,2,28175819.1628341735,None,None,/CSAFront/oidc/authorize.do?response_type=code&scope=openid+addresses+address_reg+birthdate+citizenship+education+email+gender+inn+maindoc+marital_status+mobile+name+openid+place_of_birth+previous_maindoc+previous_name+snils+verified&client_id=0916a770-fa84-464d-8457-f802ba408ef6&state=GINRH4jxUhWg&nonce=1056s0itG9Pm&redirect_uri=https://domclick.ru/cas/login,Авторизация - Сбер ID,1628961733,2021-08-14,EVENT,5,7239,SBOL_WEB_oidc,Вход по QR-коду,Получен QR,online.sberbank.ru,11002249,1628961733,False,None,desktop,Tambov Oblast,Tambov


### Load_data 

### prod_dct

In [122]:
%%time
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(SBX_TEAM_DIGITCAMP, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)").cache()

CPU times: user 2.58 ms, sys: 955 µs, total: 3.53 ms
Wall time: 75.6 ms


In [123]:
%%time
prod_dct.count()

CPU times: user 3.01 ms, sys: 184 µs, total: 3.19 ms
Wall time: 352 ms


131

### nontop

In [5]:
%%time
nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DlfTTM >= timestamp('2021-04-01'))"\
                  .format(SBX_TEAM_DIGITCAMP, 'MA_MMB_OFFER_NONTOP')).cache()

CPU times: user 2.28 ms, sys: 2.46 ms, total: 4.74 ms
Wall time: 426 ms


In [6]:
%%time
nontop.count()

CPU times: user 18.5 ms, sys: 24.9 ms, total: 43.4 ms
Wall time: 1min 57s


34327204

In [7]:
nontop.show(3)

,OFFER_ID,INN,NONTOP_FLG,PRIORITY_NUM,CRM_ID,ACTIVITY_TYPE_CD,COEF_PRODUCT_CD,NONTOP_RANG,CREATE_DTTM,OFFER_DESC,NPV_VOLUME_HEAP,NPV_INCOME_HEAP,SOURCES_LIST,SOURCES_LV2_LIST,NOTPRPR_FLG,CAMPAIGN_CODE,REASON,START_DTTM,END_DTTM,CM_CKR_FLG,CM_MS_FLG,CM_MKK_FLG,SBBOL_FLG,PRODUCT_ID,CM_GKM_FLG
0,8616143508,5079005278,2,0.0,1-2GPD7YOD,SALE,766.200012,2,2021-04-30 16:06:38,"Признак: Клиент относится к отрасли, активно присутствующей в 2ГИС.",0.0,18000.0,FILE,ЦА-2ГИСВЫВМЛ-1220,0,ЦА-2ГИСВЫВМЛ-1220,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",0,0,0,0,4344,0
1,8618440913,5079005285,2,0.0,1-2GPD7YOJ,SALE,3832.199951,1,2021-04-30 16:06:38,Клиенты с аналогичной отраслью (Деятельность религиозных организаций) часто используют данный продукт в своей деятельности.,0.0,9000.0,OTR2,OTR2,0,ЦА-РЕКОМЕНДАЦИИММБ-1020,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",0,0,0,1,298,0
2,8607890451,5079006987,2,0.0,1-12EJPMK,SALE,3832.199951,1,2021-04-30 16:06:38,Клиенты с аналогичной отраслью (Управление эксплуатацией нежилого фонда за вознаграждение или на договорной основе) часто используют данный продукт в своей деятельности.,0.0,9000.0,OTR2,OTR2,0,ЦА-РЕКОМЕНДАЦИИММБ-1020,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",0,0,0,1,298,0


In [17]:
nontop.dtypes

[('OFFER_ID', 'bigint'),
 ('INN', 'string'),
 ('NONTOP_FLG', 'string'),
 ('PRIORITY_NUM', 'float'),
 ('CRM_ID', 'string'),
 ('ACTIVITY_TYPE_CD', 'string'),
 ('COEF_PRODUCT_CD', 'float'),
 ('NONTOP_RANG', 'bigint'),
 ('CREATE_DTTM', 'timestamp'),
 ('OFFER_DESC', 'string'),
 ('NPV_VOLUME_HEAP', 'float'),
 ('NPV_INCOME_HEAP', 'float'),
 ('SOURCES_LIST', 'string'),
 ('SOURCES_LV2_LIST', 'string'),
 ('NOTPRPR_FLG', 'bigint'),
 ('CAMPAIGN_CODE', 'string'),
 ('REASON', 'string'),
 ('START_DTTM', 'timestamp'),
 ('END_DTTM', 'string'),
 ('CM_CKR_FLG', 'bigint'),
 ('CM_MS_FLG', 'bigint'),
 ('CM_MKK_FLG', 'bigint'),
 ('SBBOL_FLG', 'bigint'),
 ('PRODUCT_ID', 'bigint'),
 ('CM_GKM_FLG', 'bigint')]

есть кейсы, где потенциалы разные по одинаковым продуктам за одну дату (источники могут повторяться, а могут различаться)

То есть, при сборке потенциалов по разным продуктам, необходимо собирать сет по источникам, потенциалы по продукту при этом суммировать

In [25]:
%%time
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .show(10)

CPU times: user 13.9 ms, sys: 10.4 ms, total: 24.2 ms
Wall time: 31.7 s


,INN,PRODUCT_ID,CNT_PRD
0,3015110805,18,3
1,6659161064,24,3
2,2543132170,18,3
3,381296733734,18,3
4,2543065950,18,3
5,7736523972,18,3
6,263408599005,18,3
7,7814503581,156,3
8,5044094774,18,3
9,2360006918,18,3


In [ ]:
%%time
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

бывают кейсы, где в таблице разные потенциалы по совпадающим источникам - их несопоставимо мало, не будем обращать на них внимание

In [26]:
%%time
nontop.groupby("INN", 'PRODUCT_ID', 'SOURCES_LV2_LIST')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .show(10)

CPU times: user 15.8 ms, sys: 14.7 ms, total: 30.4 ms
Wall time: 32.4 s


,INN,PRODUCT_ID,SOURCES_LV2_LIST,CNT_PRD
0,7814503581,156,ЦА-НЕКОМАКТ-1220,2
1,5029170396,156,ЦА-НЕКОМАКТ-1220,2
2,633066266002,154,OTR2,1
3,424400047678,144,OTR2,1
4,424400047678,130,OTR2,1
5,421812909892,127,OTR2,1
6,424624106104,161,ЦА-КРЕДСМАРТ-0120,1
7,5611078708,130,OTR2,1
8,421807262539,135,OTR2,1
9,650902357007,135,OTR2,1


In [61]:
nontop.groupby("INN", 'PRODUCT_ID')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

12676

сколько строк с одним нн с одним продуктом и разными потенциалами

In [62]:
nontop.groupby("INN", 'PRODUCT_ID', 'COEF_PRODUCT_CD')\
      .agg(f.count('PRODUCT_ID').alias('CNT_PRD'))\
      .orderBy(f.col('CNT_PRD').desc())\
      .filter("CNT_PRD >= 2").count()

5930

In [59]:
nontop.filter((f.col("INN") == "263408599005") & (f.col("PRODUCT_ID") == 18)).show()

,OFFER_ID,INN,NONTOP_FLG,PRIORITY_NUM,CRM_ID,ACTIVITY_TYPE_CD,COEF_PRODUCT_CD,NONTOP_RANG,CREATE_DTTM,OFFER_DESC,NPV_VOLUME_HEAP,NPV_INCOME_HEAP,SOURCES_LIST,SOURCES_LV2_LIST,NOTPRPR_FLG,CAMPAIGN_CODE,REASON,START_DTTM,END_DTTM,CM_CKR_FLG,CM_MS_FLG,CM_MKK_FLG,SBBOL_FLG,PRODUCT_ID,CM_GKM_FLG
0,8595188595,263408599005,2,0.0,1-D47S3QX,EXPANS,5.543307e+09,1,2021-04-30 16:06:38,"У Клиента есть действующий Договор эквайринга в Сбербанке. По данным 2 ГИС обнаружен потенциал на расширение: количество действующих ТСТ Клиента больше на 2, чем подключено к Договору эквайринга (ЦА-ЭКВ2ГИСР-0121)",9427475.0,193263.234375,FILE_FA,ЦА-ЭКВ2ГИСР-0121,0,ЦА-ЭКВ2ГИСР-0121,1,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,0,1,0,18,0
1,8632869950,263408599005,2,0.0,1-D47S3QX,CHURN,5.543307e+09,2,2021-04-30 16:06:38,"ОТ15 Клиент более 11 дней перестал проводить операции Торгового Эквайринг, уточнить причины и при необходимости восстановить сотрудничество с банком. Тип устройства: Переносной POS-терминал, по адресу: улица Дзержинского, 100 (KRASNODAR). Дата последней операции: 31.03.21. Ранее клиент проводил платежи каждую неделю. Обращений клиента по вопросам работы эквайринга не было. Потенциальные потери комиссии по Эквайрингу - 59 тыс. руб. в год",9427475.0,193263.234375,FILE_FA,ЦА-ОТММБ2-1220,0,ЦА-ОТММБ2-1220,1,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,0,1,0,18,0
2,8624472350,263408599005,2,0.0,1-D47S3QX,SALE,5.543307e+03,4,2021-04-30 16:06:38,"Клиент по данным 2ГИС имеет точки с признаком «Расчет по картам/Наличный расчет».|Клиенты с аналогичной отраслью (Торговля розничная сувенирами, изделиями народных художественных промыслов) часто используют данный продукт в своей деятельности.",9427475.0,193263.234375,"FILE,OTR2","ЦА-ПРОД2ГИСММБ-0120,OTR2",0,ЦА-ПРОД2ГИСММБ-0120,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,1,1,1,18,1


In [27]:
nontop.filter((f.col("INN") == "7814503581") & (f.col("PRODUCT_ID") == 156)).show()

,OFFER_ID,INN,NONTOP_FLG,PRIORITY_NUM,CRM_ID,ACTIVITY_TYPE_CD,COEF_PRODUCT_CD,NONTOP_RANG,CREATE_DTTM,OFFER_DESC,NPV_VOLUME_HEAP,NPV_INCOME_HEAP,SOURCES_LIST,SOURCES_LV2_LIST,NOTPRPR_FLG,CAMPAIGN_CODE,REASON,START_DTTM,END_DTTM,CM_CKR_FLG,CM_MS_FLG,CM_MKK_FLG,SBBOL_FLG,PRODUCT_ID,CM_GKM_FLG
0,8612525968,7814503581,2,0.0,1-6YPIP6L,EXPANS,2936.08374,5,2021-04-30 16:06:38,"РШ8 Предложить перевод текущих оборотов на расчетный счет в Сбербанке. В течение предыдущего месяца не менее 5 контрагентов Клиента (со счетов в Сбербанке) перечислили 1 млн. руб. на счета Клиента в другие КБ. Основной банк-конкурент: ПАО ""БАНК ""САНКТ-ПЕТЕРБУРГ"": 1 млн. руб. (Сбербанк: 0 млн. руб.)",167752864.0,229310.84375,Trigger_SAS,T145,0,ЦА-РЕКОМЕНДАЦИИММБ-1020,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,0,1,0,156,0
1,8592974146,7814503581,2,0.0,1-6YPIP6L,AUTHORITY,2936.08374,6,2021-04-30 16:06:38,"20.05.2021 у клиента заканчивается срок действия полномочий руководителя. Во избежание сложностей по управлению счетом, необходимо напомнить клиенту о необходимости продлить полномочия для Сидоров Антон Вадимович",167752864.0,229310.84375,FILE,ЦА-НЕКОМАКТ-1220,0,ЦА-НЕКОМАКТ-1220,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,0,0,0,156,0
2,8613219240,7814503581,2,0.0,1-6YPIP6L,GRACE PERIOD,2936.08374,7,2021-04-30 16:06:38,"16.05.21 у клиента завершается срок действия льготы по ПУ Набирая обороты, подключенного к счету (последние цифры) *89960. Плата по пакету услуг после окончания льготы составит 990 руб./мес. Проинформируйте клиента о завершении срока действия льготы. В случае необходимости помогите подобрать новый пакет услуг.",167752864.0,229310.84375,FILE,ЦА-НЕКОМАКТ-1220,0,ЦА-НЕКОМАКТ-1220,0,2021-04-29,"java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id=""Europe/Moscow"",offset=10800000,dstSavings=0,useDaylight=false,transitions=79,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=9999,MONTH=11,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=31,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]",1,0,0,0,156,0


посмотрим, как выглядят источники в нонтопе

In [32]:
%%time
nontop.select("SOURCES_LV2_LIST").distinct().show(4)

CPU times: user 15.2 ms, sys: 1.32 ms, total: 16.5 ms
Wall time: 4.11 s


,SOURCES_LV2_LIST
0,"OTR2,ЦА-ДМСМ-0121"
1,"T020,ЦА-ПРОД2ГИСММБ-0120,ЦА-ФОТММБ-0221,NBA"
2,ЦА-ЭКВПИРР-0421
3,"T099,ЦА-ДБКМЛ-0421,NBA"


выделим все источники в список

In [29]:
source_list = nontop.select("SOURCES_LV2_LIST").distinct().collect()

In [ ]:
source_list

In [22]:
source_list[0][0].split(',')

['OTR2', 'ЦА-ДМСМ-0121']

Выделим необходимые столбцы

In [82]:
nontop_need_columns = nontop.select("INN",
                                    "PRODUCT_ID",
                                    "SOURCES_LV2_LIST",
                                    f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_NONTOP"),
                                    f.col("COEF_PRODUCT_CD").alias("POTENTIAL")).cache()

In [83]:
%%time
nontop_need_columns.count()

CPU times: user 196 µs, sys: 2.96 ms, total: 3.16 ms
Wall time: 1.44 s


34327204

In [86]:
nontop_need_columns.show()

,INN,PRODUCT_ID,SOURCES_LV2_LIST,SOURCES_LV2_LIST_NONTOP,POTENTIAL
0,5079005278,4344,ЦА-2ГИСВЫВМЛ-1220,[ЦА-2ГИСВЫВМЛ-1220],766.200012
1,5079005285,298,OTR2,[OTR2],3832.199951
2,5079006987,298,OTR2,[OTR2],3832.199951
3,5079006987,144,OTR2,[OTR2],411.839996
4,507900751574,151,OTR2,[OTR2],3929.560791
5,507900751574,18,OTR2,[OTR2],3928.852783
6,507900751574,298,OTR2,[OTR2],3832.199951
7,507900751574,124,OTR2,[OTR2],3714.423340
8,507900751574,123,OTR2,[OTR2],1911.679565
9,507900751574,135,OTR2,[OTR2],848.261292


In [85]:
nontop_need_columns.groupby("INN", "PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST"))\
                   .filter((f.col("INN") == "7814503581") & (f.col("PRODUCT_ID") == 156)).show()

,INN,PRODUCT_ID,collect_set(SOURCES_LV2_LIST)
0,7814503581,156,"[T145, ЦА-НЕКОМАКТ-1220]"


In [43]:
nontop_need_columns.select(f.col("SOURCES_LV2_LIST"),
                           f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCE_LST")).distinct().show(100)

,SOURCES_LV2_LIST,SOURCE_LST
0,"T038,T002,SEASON","[T038, T002, SEASON]"
1,"T038,T239,NBA","[T038, T239, NBA]"
2,"T036,T120,T181","[T036, T120, T181]"
3,"T080,ЦА-ПРОД2ГИСММБ-0120,OTR2,ЦА-БКПЕРЕВОД-0421","[T080, ЦА-ПРОД2ГИСММБ-0120, OTR2, ЦА-БКПЕРЕВОД-0421]"
4,"T001,T107,T120,T182","[T001, T107, T120, T182]"
5,"T038,OTR2,NBA,SEASON,ЦА-ДОМКЛИКММБ-1120","[T038, OTR2, NBA, SEASON, ЦА-ДОМКЛИКММБ-1120]"
6,"T038,ЦА-ДОМКЛИКММБ-1120,ЦА-МСККРЕД7ГП-0221,T239","[T038, ЦА-ДОМКЛИКММБ-1120, ЦА-МСККРЕД7ГП-0221, T239]"
7,"T133,T132,ЦА-ФОТСПЕЦТЕХ-0321","[T133, T132, ЦА-ФОТСПЕЦТЕХ-0321]"
8,"ЦА-КРЕДМСХ1528-0421,ЦА-МСККРЕД7ГП-0221,T239,T182","[ЦА-КРЕДМСХ1528-0421, ЦА-МСККРЕД7ГП-0221, T239, T182]"
9,"T120,OTR2,OTR2,T182","[T120, OTR2, OTR2, T182]"


### nontop pipeline

таким образом опишем весь пайплайн того, что происходит с нонтопом

In [87]:
nontop_need_columns = nontop.select(f.to_date(f.col('CREATE_DTTM')).alias('CREATE_DATE'),
                                    "INN",
                                    "PRODUCT_ID",
                                    f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_NONTOP"),
                                    f.col("COEF_PRODUCT_CD").alias("POTENTIAL_NONTOP")).cache()

In [88]:
%%time
nontop_need_columns.count()

CPU times: user 2.04 ms, sys: 1.53 ms, total: 3.57 ms
Wall time: 1.67 s


34327204

In [89]:
nontop_need_columns.show(3)

,CREATE_DATE,INN,PRODUCT_ID,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,5079005278,4344,[ЦА-2ГИСВЫВМЛ-1220],766.200012
1,2021-04-30,5079005285,298,[OTR2],3832.199951
2,2021-04-30,5079006987,298,[OTR2],3832.199951


In [117]:
%%time
nontop_all_sources = nontop_need_columns.groupby("CREATE_DATE",
                                                 "INN",
                                                 "PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST_NONTOP")\
                                                                    .alias("SOURCES_LV2_LIST_NONTOP"),
                                                                   f.avg("POTENTIAL_NONTOP").alias("POTENTIAL_NONTOP")).cache()

CPU times: user 3.13 ms, sys: 3.11 ms, total: 6.24 ms
Wall time: 28.9 ms


In [118]:
nontop_all_sources.show(5)

,CREATE_DATE,INN,PRODUCT_ID,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,010100017999,123,[[OTR2]],1.803992e+03
1,2021-04-30,010100206570,123,[[OTR2]],1.803992e+03
2,2021-04-30,010100333289,130,[[OTR2]],5.320272e+02
3,2021-04-30,010100448410,127,[[OTR2]],8.837892e+02
4,2021-04-30,010100637706,124,"[[ЦА-КРЕДМСХ1528-0421, OTR2, SEASON]]",8.822050e+09
5,2021-04-30,0101008493,123,[[OTR2]],1.803992e+03
6,2021-04-30,0101012940,123,[[OTR2]],1.803992e+03
7,2021-04-30,010101428705,19,[[OTR2]],3.016152e+03
8,2021-04-30,010101626376,123,[[OTR2]],1.803992e+03
9,2021-04-30,010101626376,124,[[OTR2]],3.634922e+03


In [113]:
def list_to_set(source_lst):
    a = set()
    for i in source_lst:
        a = a | set(i)
    return list(a)      

In [115]:
set_from_list_udf = f.udf(list_to_set, ArrayType(StringType()))

In [119]:
nontop_before_end = nontop_all_sources.withColumn("SOURCES_LV2_LIST_NONTOP",
                                                  set_from_list_udf(f.col("SOURCES_LV2_LIST_NONTOP")))

In [120]:
%%time
nontop_before_end.show()

CPU times: user 6.49 ms, sys: 2.24 ms, total: 8.73 ms
Wall time: 146 ms


,CREATE_DATE,INN,PRODUCT_ID,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,010100017999,123,[OTR2],1.803992e+03
1,2021-04-30,010100206570,123,[OTR2],1.803992e+03
2,2021-04-30,010100333289,130,[OTR2],5.320272e+02
3,2021-04-30,010100448410,127,[OTR2],8.837892e+02
4,2021-04-30,010100637706,124,"[OTR2, ЦА-КРЕДМСХ1528-0421, SEASON]",8.822050e+09
5,2021-04-30,0101008493,123,[OTR2],1.803992e+03
6,2021-04-30,0101012940,123,[OTR2],1.803992e+03
7,2021-04-30,010101428705,19,[OTR2],3.016152e+03
8,2021-04-30,010101626376,123,[OTR2],1.803992e+03
9,2021-04-30,010101626376,124,[OTR2],3.634922e+03


In [209]:
nontop_final = nontop_before_end.join(prod_dct, nontop_before_end.PRODUCT_ID == prod_dct.ID, how='inner')\
                                .select("CREATE_DATE",
                                        "INN",
                                        f.col("CRM_PRODUCT_ID").alias("CRM_PRODUCT_ID"),
                                        f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                        f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                        "SOURCES_LV2_LIST_NONTOP",
                                        "POTENTIAL_NONTOP"
                                        ).cache()

In [210]:
nontop_final.count()

33296952

In [211]:
nontop_final.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,010100017999,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
1,2021-04-30,010100206570,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
2,2021-04-30,010100333289,1-PJ1ZWPY,Страхование,Страхование,[OTR2],5.320272e+02
3,2021-04-30,010100448410,1-3BGP3MY,Страхование,Страхование,[OTR2],8.837892e+02
4,2021-04-30,010100637706,1-1GVC,Кредитование,Кредитование,"[OTR2, ЦА-КРЕДМСХ1528-0421, SEASON]",8.822050e+09
5,2021-04-30,0101008493,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
6,2021-04-30,0101012940,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
7,2021-04-30,010101428705,1-115HHYO,Бизнес-карта,Бизнес-карта,[OTR2],3.016152e+03
8,2021-04-30,010101626376,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
9,2021-04-30,010101626376,1-1GVC,Кредитование,Кредитование,[OTR2],3.634922e+03


Оставить в итоге, 


INN	CRM_PRODUCT_ID PRODUCT_GROUP	PRODUCT_SUBGROUP	CREATE_DATE	POTENTIAL, SOURCES_LV2_LIST


список источников, как фичи оставить, словарик с регулярным обновлением, если придут новые источники

In [ ]:
nontop_full_agg.union()

scaller

In [123]:
from pyspark.ml.feature import StandardScaler, VectorAssembler, MinMaxScaler

In [124]:
nontopScaller = MinMaxScaler()
nontopScaller.setInputCol('POTENTIAL_VECTOR')
nontopScaller.setOutputCol('POTENTIAL_SCALED')

MinMaxScaler_1a9364da3769

In [125]:
assembler = VectorAssembler().setInputCols(['POTENTIAL']).setOutputCol('POTENTIAL_VECTOR')

In [126]:
%%time
assembler.transform(nontop_full_agg).show()

CPU times: user 12.1 ms, sys: 4.57 ms, total: 16.7 ms
Wall time: 90.4 ms


,INN,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,CREATE_DATE,POTENTIAL,POTENTIAL_VECTOR
0,665206102666,1-1GVC,CREDIT,Оборот. кредит-е,Кредитование,Кредитование,2021-04-30,3473.368652,[3473.36865234375]
1,782516700000,1-3BGP3MY,CYBERINSUR,Страхование НС,Страхование,Страхование,2021-04-30,227.755203,[227.7552032470703]
2,6658274690,1-MP71XI8,LYURIST,Юридические решения-Юрист для бизнеса,Экосистема,Юридические решения,2021-04-30,411.839996,[411.8399963378906]
3,6658327239,1-1CA6IU4Y,RABOTA_RU,Работа ру-Платформа,Экосистема,Работа Ру,2021-04-30,1163.469604,[1163.4696044921875]
4,7321319730,1-XNQFAFZ,DELOVAYA SREDA,Деловая среда-Подключение к Платформе Деловая Среда,Экосистема,Деловая среда,2021-04-30,3832.199951,[3832.199951171875]
5,5403346561,1-Z3XKJ1V,MOBILEBUSINESSLINK,СберТелеком-Мобильная связь для бизнеса,Экосистема,СберТелеком,2021-04-30,50.912998,[50.91299819946289]
6,5249102634,1-ASDFAMH,EVOTOR,Эвотор-Покупка смарт-терминала,Смарт-терминал,Эквайринг,2021-04-30,848.261292,[848.2612915039062]
7,7728437102,1-JNGJRQ5,CRM,CRM,Экосистема,1С Битрикс,2021-04-30,2.039090,[2.0390896797180176]
8,6731048070,1-XNQFAFZ,DELOVAYA SREDA,Деловая среда-Подключение к Платформе Деловая Среда,Экосистема,Деловая среда,2021-04-30,4244.399902,[4244.39990234375]
9,667112826339,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,2021-04-30,5543.307129,[5543.30712890625]


In [127]:
%%time
scaled_nontop_full = nontopScaller.fit(assembler.transform(nontop_full_agg))\
                                  .transform(assembler.transform(nontop_full_agg))

CPU times: user 29.1 ms, sys: 37.3 ms, total: 66.4 ms
Wall time: 3.69 s


In [128]:
unlist = f.udf(lambda x: float(list(x)[0]), DoubleType())

In [129]:
scaled_nontop_full.show(3)

,INN,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,CREATE_DATE,POTENTIAL,POTENTIAL_VECTOR,POTENTIAL_SCALED
0,665206102666,1-1GVC,CREDIT,Оборот. кредит-е,Кредитование,Кредитование,2021-04-30,3473.368652,[3473.36865234375],[6.860769253361944e-08]
1,782516700000,1-3BGP3MY,CYBERINSUR,Страхование НС,Страхование,Страхование,2021-04-30,227.755203,[227.7552032470703],[4.498733224520552e-09]
2,6658274690,1-MP71XI8,LYURIST,Юридические решения-Юрист для бизнеса,Экосистема,Юридические решения,2021-04-30,411.839996,[411.8399963378906],[8.134866946165031e-09]


In [130]:
scaled_nontop_full = scaled_nontop_full.withColumn("POTENTIAL_SCALED",
                                                   unlist("POTENTIAL_SCALED"))

In [131]:
scaled_nontop_full.show(3)

,INN,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,CREATE_DATE,POTENTIAL,POTENTIAL_VECTOR,POTENTIAL_SCALED
0,665206102666,1-1GVC,CREDIT,Оборот. кредит-е,Кредитование,Кредитование,2021-04-30,3473.368652,[3473.36865234375],6.860769e-08
1,782516700000,1-3BGP3MY,CYBERINSUR,Страхование НС,Страхование,Страхование,2021-04-30,227.755203,[227.7552032470703],4.498733e-09
2,6658274690,1-MP71XI8,LYURIST,Юридические решения-Юрист для бизнеса,Экосистема,Юридические решения,2021-04-30,411.839996,[411.8399963378906],8.134867e-09


In [134]:
max_potential = scaled_nontop_full.select(f.max("POTENTIAL")).take(1)[0][0]
scaled_nontop_full.filter(f.col('POTENTIAL') == max_potential).show()

,INN,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,CREATE_DATE,POTENTIAL,POTENTIAL_VECTOR,POTENTIAL_SCALED
0,370103252404,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
1,7727619526,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
2,563800182903,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
3,5405460157,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
4,550411431048,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
5,7804499093,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
6,5903091417,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
7,0278076122,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
8,3329059980,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0
9,6950054654,1-79LMUVD,FACTORING,Факторинг,Факторинг,Факторинг,2021-04-30,5.062652e+10,[50626519040.0],1.0


не очень хорошо, что минимальный потенциал приравнивается к 0, надо сделать его не от 0 до 1 а от epsilon до 1

### OFFER_PRIORITY

In [130]:
priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('2021-04-01')"\
                    .format(SBX_TEAM_DIGITCAMP, 'OFFER_PRIORITY')).cache()

In [132]:
priority.count()

1406268

In [131]:
%%time
priority.show(3)

CPU times: user 28 ms, sys: 8.67 ms, total: 36.7 ms
Wall time: 52 s


,offer_id,run_id,load_id,load_dttm,crm_product_id,offer_priority,user_id,update_dttm,end_dt,activity_type_cd,start_dt,km_group_id,offer_desc,sources_list,sources_lv2_list,campaign_code,npv_volume_heap,npv_income_heap,inn,crm_id,product_cd,reason,cm_ckr_flg,cm_ms_flg,cm_mkk_flg,sbbol_flg,mkk_activity,product_id
0,8376141173,2671,94966,2021-04-13,1-1GUC,4461.047852,1-KNBZ10S,2021-04-13,2021-04-14,SALE,2021-04-13,1,Клиенты с аналогичной отраслью (Предоставление услуг парикмахерскими и салонами красоты) часто используют данный продукт в своей деятельности.,OTR2,OTR2,ЦА-ПОТЕНЦИАЛММБЦКР-0619,280062.0,8001.549805,510100123694,1-166G2THU,ACQUIRING,0,1.0,1.0,1.0,1.0,None,18
1,8376183666,2671,94966,2021-04-13,1-1GUC,4461.047852,1-KNBZ10S,2021-04-13,2021-04-14,SALE,2021-04-13,1,"Клиенты с аналогичной отраслью (Торговля розничная сувенирами, изделиями народных художественных промыслов) часто используют данный продукт в своей деятельности.",OTR2,OTR2,ЦА-ПОТЕНЦИАЛММБЦКР-0619,NaN,52176.000000,301509824829,1-17ZM3J5,ACQUIRING,0,1.0,1.0,1.0,1.0,None,18
2,8376840618,2671,94966,2021-04-13,1-1GUC,4461.047852,1-KNBZ10S,2021-04-13,2021-04-14,SALE,2021-04-13,1,Клиенты с аналогичной отраслью (Торговля розничная одеждой в специализированных магазинах) часто используют данный продукт в своей деятельности.,OTR2,OTR2,ЦА-ПОТЕНЦИАЛММБЦКР-0619,NaN,52176.000000,181200134430,1-1R81TUJJ,ACQUIRING,0,1.0,1.0,1.0,1.0,None,18


In [47]:
print(priority.columns)

['offer_id', 'run_id', 'load_id', 'load_dttm', 'crm_product_id', 'offer_priority', 'user_id', 'update_dttm', 'end_dt', 'activity_type_cd', 'start_dt', 'km_group_id', 'offer_desc', 'sources_list', 'sources_lv2_list', 'campaign_code', 'npv_volume_heap', 'npv_income_heap', 'inn', 'crm_id', 'product_cd', 'reason', 'cm_ckr_flg', 'cm_ms_flg', 'cm_mkk_flg', 'sbbol_flg', 'mkk_activity', 'product_id']


в приорити даты только до 6 мая??

In [51]:
%%time
priority.select('load_dttm').distinct().show(100)

CPU times: user 6 ms, sys: 6.9 ms, total: 12.9 ms
Wall time: 554 ms


,load_dttm
0,2021-04-30
1,2021-04-20
2,2021-04-22
3,2021-04-13
4,2021-04-27
5,2021-05-06
6,2021-04-06
7,2021-04-07
8,2021-04-15


в priority нет даты старта, но есть load_dttm - использую ее, как эквивалентную 

In [10]:
priority_part = priority.select('INN', 
                                'CRM_PRODUCT_ID', 
                                f.col('offer_priority').alias("POTENTIAL"),
                                f.col('load_dttm').alias("CREATE_DTTM")
                                )

In [11]:
priority_part = priority_part.join(product_dict_sp,
                                   on='CRM_PRODUCT_ID',
                                   how='inner').select('INN', 
                                                       'CRM_PRODUCT_ID', 
                                                       'PRODUCT_CD_MMB',
                                                       'PRODUCT_SHORT_NM',
                                                       'PRODUCT_GROUP',
                                                       'PRODUCT_SUBGROUP',
                                                       'POTENTIAL').filter("PRODUCT_CD_MMB is not Null")

In [15]:
%%time
priority_part.count()

CPU times: user 1.33 ms, sys: 1.58 ms, total: 2.91 ms
Wall time: 514 ms


1294404

In [14]:
%%time
priority_part.show()

CPU times: user 8.23 ms, sys: 1.5 ms, total: 9.73 ms
Wall time: 438 ms


,INN,CRM_PRODUCT_ID,PRODUCT_CD_MMB,PRODUCT_SHORT_NM,PRODUCT_GROUP,PRODUCT_SUBGROUP,POTENTIAL
0,510100123694,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
1,301509824829,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
2,181200134430,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
3,581202937402,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
4,132804491330,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,3.994407e+03
5,7727211310,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
6,520102809301,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,3.994407e+09
7,130802692781,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
8,502300829558,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,4.461048e+03
9,519010682302,1-1GUC,ACQUIRING,Торговый эквайринг,Эквайринг,Эквайринг,3.994407e+03


### priority pipeline 

In [133]:
priority_need_columns = priority.select(f.to_date(f.col('load_dttm')).alias('CREATE_DATE'),
                                        "INN",
                                        "CRM_PRODUCT_ID",
                                        f.split(f.col("SOURCES_LV2_LIST"), ',').alias("SOURCES_LV2_LIST_PRIORITY"),
                                        f.col("offer_priority").alias("POTENTIAL_PRIORITY")).cache()

In [139]:
%%time
priority_all_sources = priority_need_columns.groupby("CREATE_DATE",
                                                     "INN",
                                                     "CRM_PRODUCT_ID").agg(f.collect_set("SOURCES_LV2_LIST_PRIORITY")\
                                                                        .alias("SOURCES_LV2_LIST_PRIORITY"),
                                                                       f.avg("POTENTIAL_PRIORITY")\
                                                                        .alias("POTENTIAL_PRIORITY")).cache()

CPU times: user 2.42 ms, sys: 3.93 ms, total: 6.35 ms
Wall time: 28.3 ms


In [142]:
priority_before_end = priority_all_sources.withColumn("SOURCES_LV2_LIST_PRIORITY",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_PRIORITY")))

In [143]:
priority_before_end.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY
0,2021-04-06,0105053264,1-HGDJ1PU,"[ЦА-КОРСПКСББОЛ-0420, ЦА-КОРСПК1-0221]",7.365011e+02
1,2021-04-06,010515470405,1-1GVC,"[T181, OTR2, SEASON]",6.730691e+09
2,2021-04-06,0216003205,1-EO0QXLI,[ЦА-КРЕДСМАРТ-0120],6.687822e+09
3,2021-04-06,0242009549,1-2UI5D8E1,[ЦА-ФОРДСУБ-0321],9.186420e+11
4,2021-04-06,0263024788,1-2UI5D8E1,[ЦА-ФОРДСУБ-0321],9.186420e+11
5,2021-04-06,026303659999,1-XNQFAFZ,[OTR2],3.786300e+03
6,2021-04-06,032396792100,1-1GW0,[ЦА-ПРОД2ГИСММБ-0120],6.280826e+03
7,2021-04-06,0326046730,1-1GW0,[ЦА-ФОТММБ-0221],6.280826e+03
8,2021-04-06,032618563841,1-2UI5D8E1,[ЦА-ФОРДСББ-0321],9.279212e+11
9,2021-04-06,0516012240,1-1GW0,[ЦА-ФОТММБ-0221],5.676855e+03


In [206]:
priority_final = priority_before_end.join(prod_dct, priority_before_end.CRM_PRODUCT_ID == prod_dct.CRM_PRODUCT_ID, how='inner')\
                                .select("CREATE_DATE",
                                        "INN",
                                        priority_before_end.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                        f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                        f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                        "SOURCES_LV2_LIST_PRIORITY",
                                        f.col("POTENTIAL_PRIORITY")
                                        ).cache()

In [207]:
%%time
priority_final = priority_final.filter("CREATE_DATE = '2021-04-30'").cache()

CPU times: user 1.67 ms, sys: 1.65 ms, total: 3.32 ms
Wall time: 15.1 ms


In [208]:
priority_final.count()

311821

In [162]:
priority_final.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY
0,2021-04-30,010400309022,1-K3MF6Z8,Кредитование,Кредитная бизнес-карта,[ЦА-КРЕДСМАРТ-0120],1.810304e+04
1,2021-04-30,010503995373,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03
2,2021-04-30,0105051041,1-1GW0,Зарплатные проекты,Зарплатные проекты,[ЦА-ФОТММБ-0221],6.264109e+03
3,2021-04-30,020503617297,1-115HHZC,РКО,РКО,[ЦА-ВОСММБ-0720],2.645477e+03
4,2021-04-30,026202378462,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03
5,2021-04-30,026510823100,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03
6,2021-04-30,026816846803,1-EO0QXLI,Кредитование,Кредитование,[ЦА-КРЕДСМАРТ-0120],1.112846e+10
7,2021-04-30,0272914709,1-115HHYO,Бизнес-карта,Бизнес-карта,[OTR2],3.016152e+03
8,2021-04-30,0273931288,1-115HHZC,РКО,РКО,[ЦА-ВОСММБ-0720],2.645477e+03
9,2021-04-30,027408161306,1-ZYQO0PN,Страхование,Страхование,[T100],5.241120e+01


In [175]:
nontop_final.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP
0,2021-04-30,010100017999,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
1,2021-04-30,010100206570,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
2,2021-04-30,010100333289,1-PJ1ZWPY,Страхование,Страхование,[OTR2],5.320272e+02
3,2021-04-30,010100448410,1-3BGP3MY,Страхование,Страхование,[OTR2],8.837892e+02
4,2021-04-30,010100637706,1-1GVC,Кредитование,Кредитование,"[OTR2, ЦА-КРЕДМСХ1528-0421, SEASON]",8.822050e+09
5,2021-04-30,0101008493,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
6,2021-04-30,0101012940,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
7,2021-04-30,010101428705,1-115HHYO,Бизнес-карта,Бизнес-карта,[OTR2],3.016152e+03
8,2021-04-30,010101626376,1-HLPSULB,Кредитование,Кредитная бизнес-карта,[OTR2],1.803992e+03
9,2021-04-30,010101626376,1-1GVC,Кредитование,Кредитование,[OTR2],3.634922e+03


In [177]:
nontop_final.count()

33296952

### toxic 

### join offers 

In [214]:
%%time
result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                             .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                            nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE).alias("CREATE_DATE"),
                                     f.when(f.isnull(priority_final.INN),
                                            nontop_final.INN).otherwise(priority_final.INN).alias("INN"),
                                     f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                            nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID).alias("CRM_PRODUCT_ID"),
                                     f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                            nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP).alias("PRODUCT_GROUP"),
                                     f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                            nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP).alias("PRODUCT_SUBGROUP"),
                                     f.col("SOURCES_LV2_LIST_NONTOP"),
                                     f.col("POTENTIAL_NONTOP"),
                                     f.col("SOURCES_LV2_LIST_PRIORITY"),
                                     f.col("POTENTIAL_PRIORITY"))\
                             .cache()

CPU times: user 5.78 ms, sys: 6.36 ms, total: 12.1 ms
Wall time: 65.5 ms


In [215]:
result_table.count()

33581861

In [216]:
result_table.show()

,CREATE_DATE,INN,CRM_PRODUCT_ID,PRODUCT_GROUP,PRODUCT_SUBGROUP,SOURCES_LV2_LIST_NONTOP,POTENTIAL_NONTOP,SOURCES_LV2_LIST_PRIORITY,POTENTIAL_PRIORITY
0,2021-04-30,010100195247,1-3BGP3MY,Страхование,Страхование,[OTR2],8.837892e+02,None,None
1,2021-04-30,010100358540,1-XNQFAFZ,Экосистема,Деловая среда,"[ЦА-ДСПЛАТФМЛ-0221, OTR2]",4.244400e+03,None,None
2,2021-04-30,010100400827,1-1AFS898R,Банковская гарантия,Банковская гарантия,[ЦА-ПРЕДОДОБАСТММБ-0620],8.588580e+03,None,None
3,2021-04-30,010100600960,1-1Z85A72,Кредитование,Кредитование,[OTR2],2.450188e+03,None,None
4,2021-04-30,0101007806,1-ZYQP07F,Экосистема,СберЗдоровье,[OTR2],6.408864e+02,None,None
5,2021-04-30,0101010132,1-MP71XI8,Экосистема,Юридические решения,[OTR2],4.118400e+02,None,None
6,2021-04-30,0101010492,1-1GVC,Кредитование,Кредитование,"[OTR2, SEASON]",8.822050e+09,None,None
7,2021-04-30,010101481427,1-1GUC,Эквайринг,Эквайринг,[OTR2],3.928853e+03,None,None
8,2021-04-30,010102372195,1-K3MF6Z8,Кредитование,Кредитная бизнес-карта,[ЦА-КРЕДСМАРТ-0120],1.810304e+04,None,None
9,2021-04-30,010102787908,1-1GVC,Кредитование,Кредитование,"[OTR2, SEASON]",7.779226e+09,None,None


сделать юнион источников нонтоп и приорити

### union offers 

In [22]:
mean_priority = priority_part.groupBy('CRM_PRODUCT_ID').agg(f.mean('POTENTIAL').cast(IntegerType()).alias('MEAN_POTENTIAL'),
                                                            f.first('PRODUCT_CD_MMB').alias('PRODUCT_CD_MMB'),
                                                            f.first('PRODUCT_SHORT_NM').alias('PRODUCT_SHORT_NM'),
                                                            f.first('PRODUCT_GROUP').alias('PRODUCT_GROUP'),
                                                            f.first('PRODUCT_SUBGROUP').alias('PRODUCT_SUBGROUP')
                                                           ).orderBy(f.col("MEAN_POTENTIAL").desc())

mean_priority.cache()

DataFrame[CRM_PRODUCT_ID: string, MEAN_POTENTIAL: int, PRODUCT_CD_MMB: string, PRODUCT_SHORT_NM: string, PRODUCT_GROUP: string, PRODUCT_SUBGROUP: string]

### union offers 

In [23]:
union_offers = mean_priority.unionAll(mean_nontop).select('CRM_PRODUCT_ID', 
                                                          'PRODUCT_CD_MMB', 
                                                          'PRODUCT_SHORT_NM',
                                                          'PRODUCT_GROUP',
                                                          'PRODUCT_SUBGROUP',
                                                          'MEAN_POTENTIAL')
union_offers = union_offers.drop_duplicates()

NameError: name 'mean_nontop' is not defined